# MNIST Example Using PyTorch


In [20]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [21]:
import matplotlib.style
import matplotlib as mpl
mpl.style.use('classic')

In [22]:
# Initialize Hyper Parameters

input_size = 784       # The image size = 28 x 28 = 784
hidden_size = 500      # The number of nodes at the hidden layer
num_classes = 10       # The number of output classes. In this case, from 0 to 9
num_epochs = 15         # The number of times entire dataset is trained
batch_size = 128       # The size of input data took for one iteration
learning_rate = 0.001  # The speed of convergence

In [24]:
#Download MNIST DataSet 

train_dataset = dsets.MNIST(root='./data',
                           train=True,
                           transform=transforms.ToTensor(),
                           download=True)

test_dataset = dsets.MNIST(root='./data',
                           train=False,
                           transform=transforms.ToTensor())


In [25]:
len(train_dataset)
#train_dataset.shape

60000

In [1]:


import torch

from IPython.core.debugger import set_trace

def f(x):
    res = x*x
    
    set_trace()
    
    return res

x = torch.rand(1, requires_grad=True)
y = f(x)



> <ipython-input-1-d6d86af430df>(12)f()
     10     set_trace()
     11 
---> 12     return res
     13 
     14 x = torch.rand(1, requires_grad=True)

ipdb> x
tensor([0.1795], requires_grad=True)
ipdb> next
--Return--
tensor([0.032...MulBackward0>)
> <ipython-input-1-d6d86af430df>(12)f()
     10     set_trace()
     11 
---> 12     return res
     13 
     14 x = torch.rand(1, requires_grad=True)

ipdb> res
tensor([0.0322], grad_fn=<MulBackward0>)
ipdb> y
*** NameError: name 'y' is not defined
ipdb> exit()


BdbQuit: 

In [27]:
type(train_dataset)

torchvision.datasets.mnist.MNIST

In [28]:
len(test_dataset)

10000

In [29]:
# Load the DataSet

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)



In [30]:
# Create the Feed Forward NN

class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()                    # Inherited from the parent class nn.Module
        self.fc1 = nn.Linear(input_size, hidden_size)  # 1st Full-Connected Layer: 784 (input data) -> 500 (hidden node)
        self.relu = nn.ReLU()                          # Non-Linear ReLU Layer: max(0,x)
        self.fc2 = nn.Linear(hidden_size, num_classes) # 2nd Full-Connected Layer: 500 (hidden node) -> 10 (output class)
    
    def forward(self, x):                              # Forward pass: stacking each layer together
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out


In [31]:
# Instantiate the FNN

net = Net(input_size, hidden_size, num_classes)


In [32]:
# Loss Function and Optimizer

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [33]:
# Train Model

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):   # Load a batch of images with its (index, data, class)
        images = Variable(images.view(-1, 28*28))         # Convert torch tensor to Variable: change image from a vector of size 784 to a matrix of 28 x 28
        labels = Variable(labels)
        
        optimizer.zero_grad()                             # Intialize the hidden weight to all zeros
        outputs = net(images)                             # Forward pass: compute the output class given a image
        loss = criterion(outputs, labels)                 # Compute the loss: difference between the output class and the pre-given label
        loss.backward()                                   # Backward pass: compute the weight
        optimizer.step()                                  # Optimizer: update the weights of hidden nodes
        
        if (i+1) % 100 == 0:                              # Logging
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.item()))


Epoch [1/15], Step [100/468], Loss: 0.3142
Epoch [1/15], Step [200/468], Loss: 0.1618
Epoch [1/15], Step [300/468], Loss: 0.2046
Epoch [1/15], Step [400/468], Loss: 0.1928
Epoch [2/15], Step [100/468], Loss: 0.1310
Epoch [2/15], Step [200/468], Loss: 0.0879
Epoch [2/15], Step [300/468], Loss: 0.0840
Epoch [2/15], Step [400/468], Loss: 0.0592
Epoch [3/15], Step [100/468], Loss: 0.0818
Epoch [3/15], Step [200/468], Loss: 0.1235
Epoch [3/15], Step [300/468], Loss: 0.0345
Epoch [3/15], Step [400/468], Loss: 0.0695
Epoch [4/15], Step [100/468], Loss: 0.1257
Epoch [4/15], Step [200/468], Loss: 0.0861
Epoch [4/15], Step [300/468], Loss: 0.0166
Epoch [4/15], Step [400/468], Loss: 0.0728
Epoch [5/15], Step [100/468], Loss: 0.0220
Epoch [5/15], Step [200/468], Loss: 0.0569
Epoch [5/15], Step [300/468], Loss: 0.0440
Epoch [5/15], Step [400/468], Loss: 0.0313
Epoch [6/15], Step [100/468], Loss: 0.0153
Epoch [6/15], Step [200/468], Loss: 0.0353
Epoch [6/15], Step [300/468], Loss: 0.0099
Epoch [6/15

In [35]:
# Test the FNN

correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28))
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)  # Choose the best class from the output: The class with the best score
    total += labels.size(0)                    # Increment the total count
    correct += (predicted == labels).sum()     # Increment the correct count
    
print('Accuracy of the network on the 10K test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10K test images: 98 %


In [15]:
# Save the model
#torch.save(net.state_dict(), 'fnn_model.pkl')